In [1]:
!pip install ensemble_boxes

In [1]:
import pandas as pd
from ensemble_boxes import *
import numpy as np
from pycocotools.coco import COCO

In [2]:
# ensemble csv files
submission_files = ['0.5078all_swinb.csv','0.5146all_swint.csv','0.5218all_swins.csv']
submission_df = [pd.read_csv(file) for file in submission_files]

In [3]:
image_ids = submission_df[0]['image_id'].tolist()

In [4]:
annotation = '/opt/ml/input/data/test.json'
coco = COCO(annotation)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [15]:
prediction_strings = []
file_names = []
iou_thr = 0.5
skip_box_thr = 0.0001
for i, image_id in enumerate(image_ids):
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()
        if len(predict_list)==0 or len(predict_list)==1:
            continue
        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []
        for box in predict_list[:, 2:6].tolist():
            box[0] = float(box[0]) / image_info['width']
            box[1] = float(box[1]) / image_info['height']
            box[2] = float(box[2]) / image_info['width']
            box[3] = float(box[3]) / image_info['height']
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))
    
    if len(boxes_list):
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        for box, score, label in zip(boxes, scores, labels):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0] * image_info['width']) + ' ' + str(box[1] * image_info['height']) + ' ' + str(box[2] * image_info['width']) + ' ' + str(box[3] * image_info['height']) + ' '
    
    prediction_strings.append(prediction_string)
    file_names.append(image_id)

In [16]:
submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv('submission_ensemble.csv')

submission.head()

,PredictionString,image_id
0,8 0.9997581839561462 174.1595458984375 234.812...,batch_01_vt/0021.jpg
1,8 0.999539852142334 158.93875122070312 59.9969...,batch_01_vt/0028.jpg
2,1 0.9967257976531982 69.95643615722656 169.606...,batch_01_vt/0031.jpg
3,8 0.993987500667572 34.56770324707031 122.1411...,batch_01_vt/0032.jpg
4,8 0.9996205568313599 44.00043487548828 59.8103...,batch_01_vt/0070.jpg
